In [7]:
! mkdir -p data && cd data && wget -O data.zip https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-v1.zip && unzip data.zip && rm data.zip

--2020-05-06 21:34:03--  https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-v1.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.88.5
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.88.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190229076 (181M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 181.42M  14.3MB/s    in 13s     

2020-05-06 21:34:16 (14.1 MB/s) - ‘data.zip’ saved [190229076/190229076]

Archive:  data.zip
   creating: wikitext-103/
  inflating: wikitext-103/wiki.test.tokens  
  inflating: wikitext-103/wiki.valid.tokens  
  inflating: wikitext-103/wiki.train.tokens  


In [1]:
from u import *

import os
import torch

from collections import Counter

class Corpus:
    def __init__(self, word2idx, path):
        self.word2idx = word2idx
        self.train = self.tokenize(path % 'train')
        self.valid = self.tokenize(path % 'valid')
        self.test = self.tokenize(path % 'test')

    def tokenize(self, path):
        tokens = []
        with open(path, 'r') as f:
            for line in f:
                words = line.split() + ['<eos>']
                for w in words:
                    tokens.append(self.word2idx[w])
        return np.array(tokens)

%load_ext autoreload
%autoreload 2

In [5]:
# For compatibility, we use a precomputed vocab sorted from most to least occurence in the training set
idx2word = (Cache / 'sorted_vocab.npy').load()
word2idx = {w: i for i, w in enumerate(idx2word)}
assert len(word2idx) == 267735
# Might take 5-10 minutes to run
corpus = Corpus(word2idx, Data / 'wikitext-103/wiki.%s.tokens')

In [6]:
train_counts = Counter(corpus.train)
for i, (token, count) in enumerate(train_counts.most_common()): # Check that our vocab is indeed sorted
    assert count == train_counts[i]

In [17]:
for k in 'train', 'valid', 'test':
    ((Cache / 'wikitext-103').mk() / 'sorted_' + k + '.npy').save(getattr(corpus, k).astype(np.int32))